In [1]:
import multiprocessing
import torch
from psutil import virtual_memory

ram_gb = round(virtual_memory().total / 1024**3, 1)

print('CPU:', multiprocessing.cpu_count())
print('RAM GB:', ram_gb)
print("PyTorch version:", torch.__version__)
print("CUDA version:", torch.version.cuda)
print("cuDNN version:", torch.backends.cudnn.version())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device.type)

!nvidia-smi

CPU: 12
RAM GB: 15.8
PyTorch version: 2.1.0+cpu
CUDA version: None
cuDNN version: None
device: cpu
Sun Oct 29 17:28:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 442.23       Driver Version: 442.23       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 166... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   61C    P0    27W /  N/A |    154MiB /  6144MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+--------------------------------------------------------------------

In [3]:
%%capture
!pip install git+https://github.com/cene555/ru-clip-tiny.git
!gdown -O ru-clip-tiny.pkl https://drive.google.com/uc?id=1-3g3J90pZmHo9jbBzsEmr7ei5zm3VXOL
!gdown -O cifar100classes.json https://drive.google.com/uc?id=1ZJhOp5RP5wY31TyJgGNXa9xaJO8SdAhf

In [14]:
import torch
from torchvision import transforms
import transformers
from transformers import BertTokenizer
from rucliptiny import RuCLIPtiny
from predictor import Predictor
from PIL import ImageCms
from utils import get_transform
from tokenizer import Tokenizer
import cv2

In [15]:
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

In [1]:
import time

def speed_test(func, data_gen, n=5, empty_cache=True, is_text=False):
    if empty_cache: torch.cuda.empty_cache()
    values = []
    for _ in range(n):
        if is_text:
            input_data1, input_data2 = data_gen()
        else:
            input_data = data_gen()
        t = time.time()
        if is_text:
            func(input_data1, input_data2)
        else:
            func(input_data)
        values.append(time.time() - t)
        if empty_cache: torch.cuda.empty_cache()
    return sum(values) / n

In [79]:
import pandas as pd
import os
import json
from tqdm.auto import tqdm
import torch


# def prepare_classes(classes):
#     return [f"{label.lower()}" for label in classes]


# def get_text_probs_from_dataset(
#         model,
#         tokenizer=None,
#         ds=None,
#         train=False,
#         classes_path=None,
#         text_descriptions=None,
#         transform=None
# ):
#     if ds is None:
#         cls = CIFAR100
#         ds = cls(os.path.expanduser("~/.cache"), download=True, train=train)
#         if classes_path is None:
#             classes_path = f"../clip/evaluate/{name.lower()}/{name.lower()}classes.json"
#         with open(classes_path, "r") as file:
#             ds.classes = json.load(file)
#     if text_descriptions is None:
#         text_descriptions = prepare_classes(ds.classes)
#     text_input = tokenizer.tokenize(text_descriptions).to(device)
#     probs = []
#     labels = []
#     with torch.no_grad():
#         text_features = model.encode_text(text_input[0], text_input[1]).float()
#         text_features /= text_features.norm(dim=-1, keepdim=True)
#         for x in tqdm(ds, total=len(ds)):
#             images = [transform(x[0])]
#             image_features = model.encode_image(torch.stack(images).to(device)).float()
#             image_features /= image_features.norm(dim=-1, keepdim=True)
#             text_probs = (1 * image_features @ text_features.T).softmax(dim=-1)
#             labels.append(x[1])
#             probs.append(text_probs.tolist()[0])
#     return probs, labels



root_f = "C:/Users/User/anaconda3/NEURO/cars/car_data/train"

carbase = pd.DataFrame(columns=['Cars', 'Pictures'])
for f in os.listdir(root_f):
    folders = os.path.normpath(os.path.join(root_f, f))
    
    #print(f)
    file_paths = ''
    pictures = os.listdir(folders)
    data=''
    for p in pictures:
        file_path = os.path.normpath(os.path.join(folders, p))
        data+=file_path
        data+=','
    carbase.loc[len(carbase)] = [f, data]

carbase.to_csv('carbase.csv', index=False)
    





    


# def get_topk_accuracy(labels, probs, k=5):
#     successes = 0
#     for lbl, p in zip(labels, np.array(probs)):
#         _, top_labels = torch.tensor(p).topk(k, dim=-1)
#         if lbl in top_labels:
#             successes += 1
#     return successes / len(labels)